In [1]:
# # load drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install ucimlrepo

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dist
from torch.distributions.independent import Independent
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from utils_models import *
from utils_data_baselines import *

In [ ]:
def train(model, X, X_v, X_train, S_train, X_val, S_val, batch_size=128, num_epochs=100, n_samples=1, learning_rate=1e-3, patience = 50, device = "cpu", missing="selfmask"):
    # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    N = X_train.shape[0]

    train_loss_history = []
    val_loss_history = []
    train_rsme_history = []
    val_rsme_history = []
    best_val_loss = float('inf')
    best_model_state = None
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        p = np.random.permutation(N)
        X_train = X_train[p,:]
        S_train = S_train[p,:]

        for i in range(0, len(X_train), batch_size):
            x_batch = X_train[i:i+batch_size]
            s_batch = S_train[i:i+batch_size]

            optimizer.zero_grad()
            q_mu, q_log_sig2, p_mu, p_std, log_prob_z_given_x, log_prob_z, log_prob_x_given_z, log_prob_s_given_x = model(x_batch, s_batch, n_samples)

            # Calculate loss
            loss = not_miwae_loss(log_prob_z_given_x, log_prob_z, log_prob_x_given_z, log_prob_s_given_x, device=device)

            loss.backward()
            optimizer.step()

            train_loss += loss.item() * x_batch.size(0)

        train_loss /= len(X_train)
        train_loss_history.append(train_loss)

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            for i in range(0, len(X_val), batch_size):
                x_batch_val = X_val[i:i+batch_size]
                s_batch_val = S_val[i:i+batch_size]
                q_mu, q_log_sig2, p_mu, p_std, log_prob_z_given_x, log_prob_z, log_prob_x_given_z, log_prob_s_given_x = model(x_batch_val, s_batch_val, n_samples)
                val_loss += not_miwae_loss(log_prob_z_given_x, log_prob_z, log_prob_x_given_z, log_prob_s_given_x, device=device).item() * x_batch_val.size(0)
            val_loss /= len(X_val)
            val_loss_history.append(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        # get rsme for train and val
        if epoch % 10 == 0:
          train_rmse, _ = rmse_imputation(X, X_train, S_train, model, nb_samples=1_000, device=device)
          val_rmse, _ = rmse_imputation(X_v, X_val, S_val, model, nb_samples=1_000, device=device)
          train_rsme_history.append(train_rmse.item())
          val_rsme_history.append(val_rmse.item())

    return train_loss_history, val_loss_history, train_rsme_history, val_rsme_history

In [9]:
def run_experiment(missing_process="selfmask", dataset_id = 17, runs = 1, activation="tanh", path="drive/MyDrive/PGM-NotMIWAE/models"):
    # existing_models = os.listdir(path)

    # if f"modelNMW_{dataset_id}_{missing_process}_{activation}.pth" in existing_models:
    #     model = torch.load(f"{path}/modelNMW_{dataset_id}_{missing_process}_{activation}.pth")
    # else:

    train_loss_histories = []
    val_loss_histories = []
    train_rsme_histories = []
    val_rsme_histories = []
    rsme_list = []
    for run in range(runs):
        print(f"Run {run+1}/{runs}")
        X, y, (X_train_missing, mask_train, X_val_missing, mask_val, y_train, y_val) = load_and_preprocess_data(id=dataset_id)
        d = X_train_missing.shape[1] # Input dimension

        n_hidden = 128
        n_latent = d-1
        n_samples = 20    # Importance samples
        batch_size = 16      # Batch size
        # num_iterations = 600        # Number of iterations
        # num_epochs = num_iterations // (X_train_missing.shape[0] // batch_size)   # Number of epochs
        num_epochs = 300

        print(type(num_epochs))
        learning_rate = 1e-3   # Learning rate
        model = notMIWAE(d, n_latent, [n_hidden, n_hidden], [n_hidden, n_hidden], activation=nn.Tanh(), missing_process=missing_process)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        X_train_missing = X_train_missing.to(device)
        mask_train = mask_train.to(device)
        X_val_missing = X_val_missing.to(device)
        mask_val = mask_val.to(device)
        # y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
        # y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
        # X = X.to(device)
        # y = y.to(device)
        train_loss_history, val_loss_history, train_rsme_history, val_rsme_history = train(model, X, X, X_train_missing, mask_train, X_val_missing, mask_val, batch_size, num_epochs, n_samples, learning_rate, patience = 5000, device=device)
        # torch.save(model, f"{path}/modelNMW_{dataset_id}_{missing_process}_{activation}.pth")

        train_loss_histories.append(train_loss_history)
        val_loss_histories.append(val_loss_history)
        train_rsme_histories.append(train_rsme_history)
        val_rsme_histories.append(val_rsme_history)

        # calculate the rsme
        rsme = rmse_imputation(X, X_train_missing, mask_train, model, nb_samples=1_000, device=device)[0]
        rsme_list.append(rsme)

    return train_loss_histories, val_loss_histories, train_rsme_histories, val_rsme_histories, rsme_list

In [12]:
for id in [17,186, 110, 267]:
  train_loss_histories, val_loss_histories, train_rsme_histories, val_rsme_histories, rsme_list = run_experiment(runs=5, dataset_id=id, activation="tanh", missing_process="selfmask")
  print(f"Dataset {id} done")
  # print rseme_mean and std
  # convert to list
  rsme_list = [rsme.item() for rsme in rsme_list]
  print(f"Rsme mean: {np.mean(rsme_list)}")
  print(f"Rsme std: {np.std(rsme_list)}")

Run 1/5
<class 'int'>
Epoch 1/300, Train Loss: 38.9347, Val Loss: 31.4969


<ipython-input-6-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-6-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 28.9012, Val Loss: 27.3010
Epoch 3/300, Train Loss: 26.1447, Val Loss: 24.7492
Epoch 4/300, Train Loss: 25.4351, Val Loss: 24.4497
Epoch 5/300, Train Loss: 23.2834, Val Loss: 22.2648
Epoch 6/300, Train Loss: 21.7354, Val Loss: 21.8684
Epoch 7/300, Train Loss: 20.7312, Val Loss: 20.1128
Epoch 8/300, Train Loss: 19.3248, Val Loss: 20.2534
Epoch 9/300, Train Loss: 20.4968, Val Loss: 19.0981
Epoch 10/300, Train Loss: 17.2523, Val Loss: 17.7225
Epoch 11/300, Train Loss: 15.4272, Val Loss: 17.2011
Epoch 12/300, Train Loss: 15.4625, Val Loss: 15.8154
Epoch 13/300, Train Loss: 15.3518, Val Loss: 15.7155
Epoch 14/300, Train Loss: 14.2345, Val Loss: 14.3876
Epoch 15/300, Train Loss: 13.8897, Val Loss: 14.8319
Epoch 16/300, Train Loss: 12.6379, Val Loss: 13.6980
Epoch 17/300, Train Loss: 10.9915, Val Loss: 12.7025
Epoch 18/300, Train Loss: 11.7778, Val Loss: 11.0709
Epoch 19/300, Train Loss: 10.1153, Val Loss: 11.1790
Epoch 20/300, Train Loss: 9.3951, Val Loss: 10.5433
Ep

<ipython-input-6-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-6-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 35.1529, Val Loss: 32.2239
Epoch 3/300, Train Loss: 29.1797, Val Loss: 28.0919
Epoch 4/300, Train Loss: 26.7814, Val Loss: 25.1370
Epoch 5/300, Train Loss: 23.4123, Val Loss: 23.8601
Epoch 6/300, Train Loss: 22.8279, Val Loss: 22.5846
Epoch 7/300, Train Loss: 22.3476, Val Loss: 21.7677
Epoch 8/300, Train Loss: 21.1491, Val Loss: 22.2684
Epoch 9/300, Train Loss: 21.1702, Val Loss: 21.5637
Epoch 10/300, Train Loss: 19.1124, Val Loss: 19.0974
Epoch 11/300, Train Loss: 19.6652, Val Loss: 19.5163
Epoch 12/300, Train Loss: 18.1575, Val Loss: 18.9994
Epoch 13/300, Train Loss: 17.5022, Val Loss: 18.1945
Epoch 14/300, Train Loss: 15.4211, Val Loss: 17.1552
Epoch 15/300, Train Loss: 15.4182, Val Loss: 15.7346
Epoch 16/300, Train Loss: 13.8183, Val Loss: 15.2923
Epoch 17/300, Train Loss: 13.7713, Val Loss: 13.7185
Epoch 18/300, Train Loss: 11.3547, Val Loss: 12.6565
Epoch 19/300, Train Loss: 11.0221, Val Loss: 12.1620
Epoch 20/300, Train Loss: 11.4023, Val Loss: 11.8956
E

...

Epoch 2/300, Train Loss: 4.6789, Val Loss: 4.5472
Epoch 3/300, Train Loss: 4.5895, Val Loss: 4.2845
Epoch 4/300, Train Loss: 4.5143, Val Loss: 4.4130
Epoch 5/300, Train Loss: 4.4133, Val Loss: 4.2329
Epoch 6/300, Train Loss: 4.2364, Val Loss: 4.1105
Epoch 7/300, Train Loss: 4.2434, Val Loss: 4.1579
Epoch 8/300, Train Loss: 3.9850, Val Loss: 4.0463
Epoch 9/300, Train Loss: 3.8807, Val Loss: 3.8492
Epoch 10/300, Train Loss: 3.8172, Val Loss: 3.7323
Epoch 11/300, Train Loss: 3.7239, Val Loss: 3.4760
Epoch 12/300, Train Loss: 3.7323, Val Loss: 3.4895
Epoch 13/300, Train Loss: 3.4166, Val Loss: 3.4118
Epoch 14/300, Train Loss: 3.5687, Val Loss: 3.3531
Epoch 15/300, Train Loss: 3.3589, Val Loss: 3.2917
Epoch 16/300, Train Loss: 3.3170, Val Loss: 3.4182
Epoch 17/300, Train Loss: 3.3179, Val Loss: 3.2979
Epoch 18/300, Train Loss: 3.2444, Val Loss: 3.1564
Epoch 19/300, Train Loss: 3.0944, Val Loss: 2.9955
Epoch 20/300, Train Loss: 3.3608, Val Loss: 3.6200
Epoch 21/300, Train Loss: 3.2663, Val L

<ipython-input-6-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-6-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 4.9469, Val Loss: 4.7710
Epoch 3/300, Train Loss: 4.8040, Val Loss: 4.6116
Epoch 4/300, Train Loss: 4.6299, Val Loss: 4.4436
Epoch 5/300, Train Loss: 4.6215, Val Loss: 4.2612
Epoch 6/300, Train Loss: 4.3601, Val Loss: 4.1277
Epoch 7/300, Train Loss: 4.3227, Val Loss: 5.0002
Epoch 8/300, Train Loss: 4.3895, Val Loss: 4.1392
Epoch 9/300, Train Loss: 4.0695, Val Loss: 4.0009
Epoch 10/300, Train Loss: 3.8288, Val Loss: 3.6279
Epoch 11/300, Train Loss: 3.7196, Val Loss: 3.5800
Epoch 12/300, Train Loss: 3.5894, Val Loss: 3.6360
Epoch 13/300, Train Loss: 3.4808, Val Loss: 3.3973
Epoch 14/300, Train Loss: 3.4264, Val Loss: 3.6552
Epoch 15/300, Train Loss: 3.4185, Val Loss: 3.5611
Epoch 16/300, Train Loss: 3.1709, Val Loss: 3.1000
Epoch 17/300, Train Loss: 3.2381, Val Loss: 3.1254
Epoch 18/300, Train Loss: 3.2328, Val Loss: 3.2067
Epoch 19/300, Train Loss: 3.2916, Val Loss: 3.1719
Epoch 20/300, Train Loss: 3.1072, Val Loss: 3.0775
Epoch 21/300, Train Loss: 2.9361, Val L

<ipython-input-6-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-6-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 4.1901, Val Loss: 4.0929
Epoch 3/300, Train Loss: 3.8482, Val Loss: 3.6242
Epoch 4/300, Train Loss: 3.6510, Val Loss: 3.6970
Epoch 5/300, Train Loss: 3.7184, Val Loss: 3.4359
Epoch 6/300, Train Loss: 3.4459, Val Loss: 3.3644
Epoch 7/300, Train Loss: 3.4024, Val Loss: 3.3126
Epoch 8/300, Train Loss: 3.3497, Val Loss: 3.2796
Epoch 9/300, Train Loss: 3.3235, Val Loss: 3.2922
Epoch 10/300, Train Loss: 3.3000, Val Loss: 3.2617
Epoch 11/300, Train Loss: 3.1950, Val Loss: 3.3938
Epoch 12/300, Train Loss: 3.2039, Val Loss: 3.2327
Epoch 13/300, Train Loss: 3.0198, Val Loss: 2.9848
Epoch 14/300, Train Loss: 3.1330, Val Loss: 3.1877
Epoch 15/300, Train Loss: 2.9537, Val Loss: 2.8659
Epoch 16/300, Train Loss: 2.9784, Val Loss: 3.1286
Epoch 17/300, Train Loss: 2.8971, Val Loss: 2.9081
Epoch 18/300, Train Loss: 3.0686, Val Loss: 2.9953
Epoch 19/300, Train Loss: 3.0068, Val Loss: 3.0278
Epoch 20/300, Train Loss: 2.9335, Val Loss: 2.9584
Epoch 21/300, Train Loss: 2.9215, Val L

In [32]:
for id in [17,186, 110, 267]:
  train_loss_histories, val_loss_histories, train_rsme_histories, val_rsme_histories, rsme_list = run_experiment(runs=5, dataset_id=id, activation="tanh", missing_process="selfmask_known")
  print(f"Dataset {id} done")
  # print rseme_mean and std
  # convert to list
  rsme_list = [rsme.item() for rsme in rsme_list]
  print(f"Rsme mean: {np.mean(rsme_list)}")
  print(f"Rsme std: {np.std(rsme_list)}")

Run 1/5
<class 'int'>
Epoch 1/300, Train Loss: 37.0319, Val Loss: 32.1123


<ipython-input-6-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-6-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 34.1353, Val Loss: 33.8345
Epoch 3/300, Train Loss: 31.5750, Val Loss: 32.3448
Epoch 4/300, Train Loss: 30.6615, Val Loss: 31.9604
Epoch 5/300, Train Loss: 29.5998, Val Loss: 31.2564
Epoch 6/300, Train Loss: 30.3507, Val Loss: 30.5646
Epoch 7/300, Train Loss: 29.8743, Val Loss: 30.7255
Epoch 8/300, Train Loss: 29.2617, Val Loss: 30.0825
Epoch 9/300, Train Loss: 28.7827, Val Loss: 29.6419
Epoch 10/300, Train Loss: 27.6444, Val Loss: 29.3317
Epoch 11/300, Train Loss: 28.8216, Val Loss: 30.5416
Epoch 12/300, Train Loss: 28.6550, Val Loss: 29.8338
Epoch 13/300, Train Loss: 29.7036, Val Loss: 29.5421
Epoch 14/300, Train Loss: 28.4619, Val Loss: 28.7273
Epoch 15/300, Train Loss: 28.7630, Val Loss: 28.6959
Epoch 16/300, Train Loss: 28.2369, Val Loss: 29.0752
Epoch 17/300, Train Loss: 29.1213, Val Loss: 29.3282
Epoch 18/300, Train Loss: 27.6011, Val Loss: 28.4449
Epoch 19/300, Train Loss: 26.9365, Val Loss: 28.0431
Epoch 20/300, Train Loss: 26.0618, Val Loss: 28.3788
E

<ipython-input-6-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-6-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 34.2828, Val Loss: 33.6707
Epoch 3/300, Train Loss: 32.2030, Val Loss: 31.9626
Epoch 4/300, Train Loss: 32.8860, Val Loss: 30.7966
Epoch 5/300, Train Loss: 29.4959, Val Loss: 30.3484
Epoch 6/300, Train Loss: 29.8363, Val Loss: 29.5727
Epoch 7/300, Train Loss: 27.3623, Val Loss: 29.7301
Epoch 8/300, Train Loss: 29.0689, Val Loss: 29.7721
Epoch 9/300, Train Loss: 28.5076, Val Loss: 31.1140
Epoch 10/300, Train Loss: 28.6285, Val Loss: 28.3571
Epoch 11/300, Train Loss: 27.3481, Val Loss: 28.3368
Epoch 12/300, Train Loss: 29.2825, Val Loss: 29.1269
Epoch 13/300, Train Loss: 26.9113, Val Loss: 28.5791
Epoch 14/300, Train Loss: 25.8418, Val Loss: 29.5888
Epoch 15/300, Train Loss: 29.3225, Val Loss: 28.5380
Epoch 16/300, Train Loss: 26.9139, Val Loss: 28.3782
Epoch 17/300, Train Loss: 26.0015, Val Loss: 28.1955
Epoch 18/300, Train Loss: 26.5358, Val Loss: 27.6127
Epoch 19/300, Train Loss: 26.4702, Val Loss: 27.3465
Epoch 20/300, Train Loss: 27.8367, Val Loss: 28.3321
E

...

<ipython-input-6-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-6-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 25.6171, Val Loss: 26.6995
Epoch 3/300, Train Loss: 24.4483, Val Loss: 25.7446
Epoch 4/300, Train Loss: 23.0991, Val Loss: 24.3891
Epoch 5/300, Train Loss: 22.4048, Val Loss: 23.6120
Epoch 6/300, Train Loss: 21.6168, Val Loss: 23.0003
Epoch 7/300, Train Loss: 20.4913, Val Loss: 22.0540
Epoch 8/300, Train Loss: 19.4736, Val Loss: 21.0372
Epoch 9/300, Train Loss: 18.7474, Val Loss: 21.0056
Epoch 10/300, Train Loss: 18.2301, Val Loss: 19.9884
Epoch 11/300, Train Loss: 17.2903, Val Loss: 19.4181
Epoch 12/300, Train Loss: 17.6977, Val Loss: 19.7104
Epoch 13/300, Train Loss: 16.8301, Val Loss: 19.3032
Epoch 14/300, Train Loss: 16.3967, Val Loss: 18.7069
Epoch 15/300, Train Loss: 16.0964, Val Loss: 18.2382
Epoch 16/300, Train Loss: 15.1574, Val Loss: 17.8080
Epoch 17/300, Train Loss: 14.6686, Val Loss: 17.4866
Epoch 18/300, Train Loss: 14.5732, Val Loss: 17.1294
Epoch 19/300, Train Loss: 13.8498, Val Loss: 16.8911
Epoch 20/300, Train Loss: 13.4659, Val Loss: 16.6214
E

<ipython-input-6-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-6-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 24.9177, Val Loss: 25.8577
Epoch 3/300, Train Loss: 23.8370, Val Loss: 25.0804
Epoch 4/300, Train Loss: 22.6636, Val Loss: 24.0315
Epoch 5/300, Train Loss: 22.1088, Val Loss: 23.3050
Epoch 6/300, Train Loss: 21.2319, Val Loss: 22.5071
Epoch 7/300, Train Loss: 20.3531, Val Loss: 22.1023
Epoch 8/300, Train Loss: 19.4298, Val Loss: 21.1444
Epoch 9/300, Train Loss: 18.7034, Val Loss: 20.3351
Epoch 10/300, Train Loss: 17.9982, Val Loss: 20.0243
Epoch 11/300, Train Loss: 17.6286, Val Loss: 19.3056
Epoch 12/300, Train Loss: 17.2357, Val Loss: 19.5648
Epoch 13/300, Train Loss: 16.6572, Val Loss: 18.9417
Epoch 14/300, Train Loss: 16.4458, Val Loss: 18.6535
Epoch 15/300, Train Loss: 15.8254, Val Loss: 17.9558
Epoch 16/300, Train Loss: 14.9804, Val Loss: 17.8186
Epoch 17/300, Train Loss: 14.6193, Val Loss: 17.4248
Epoch 18/300, Train Loss: 14.0676, Val Loss: 17.1676
Epoch 19/300, Train Loss: 13.8334, Val Loss: 16.7775
Epoch 20/300, Train Loss: 13.4917, Val Loss: 16.3206
E

ConnectionError: Error connecting to server

In [12]:
for id in [110, 267]:
  train_loss_histories, val_loss_histories, train_rsme_histories, val_rsme_histories, rsme_list = run_experiment(runs=5, dataset_id=id, activation="tanh", missing_process="selfmask_known")
  print(f"Dataset {id} done")
  # print rseme_mean and std
  # convert to list
  rsme_list = [rsme.item() for rsme in rsme_list]
  print(f"Rsme mean: {np.mean(rsme_list)}")
  print(f"Rsme std: {np.std(rsme_list)}")

Run 1/5
<class 'int'>
Epoch 1/300, Train Loss: 7.7205, Val Loss: 6.0807


<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 5.0160, Val Loss: 9.6241
Epoch 3/300, Train Loss: 5.9327, Val Loss: 6.0542
Epoch 4/300, Train Loss: 6.2173, Val Loss: 6.7360
Epoch 5/300, Train Loss: 5.1983, Val Loss: 4.5051
Epoch 6/300, Train Loss: 4.9909, Val Loss: 4.8168
Epoch 7/300, Train Loss: 5.0586, Val Loss: 5.0817
Epoch 8/300, Train Loss: 4.5624, Val Loss: 3.6831
Epoch 9/300, Train Loss: 4.2639, Val Loss: 3.3235
Epoch 10/300, Train Loss: 3.8197, Val Loss: 3.0638
Epoch 11/300, Train Loss: 5.3649, Val Loss: 4.7945
Epoch 12/300, Train Loss: 4.1949, Val Loss: 6.6423
Epoch 13/300, Train Loss: 3.8576, Val Loss: 4.3447
Epoch 14/300, Train Loss: 5.4914, Val Loss: 4.2466
Epoch 15/300, Train Loss: 3.2990, Val Loss: 3.6484
Epoch 16/300, Train Loss: 2.7548, Val Loss: 2.9905
Epoch 17/300, Train Loss: 2.4315, Val Loss: 1.7908
Epoch 18/300, Train Loss: 2.7223, Val Loss: 1.9401
Epoch 19/300, Train Loss: 2.3258, Val Loss: 1.9271
Epoch 20/300, Train Loss: 2.0006, Val Loss: 1.9244
Epoch 21/300, Train Loss: 2.7604, Val L

<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 4.0757, Val Loss: 4.1828
Epoch 3/300, Train Loss: 4.3019, Val Loss: 6.6248
Epoch 4/300, Train Loss: 4.1551, Val Loss: 5.7894
Epoch 5/300, Train Loss: 3.2706, Val Loss: 3.9039
Epoch 6/300, Train Loss: 2.8167, Val Loss: 4.9653
Epoch 7/300, Train Loss: 3.0552, Val Loss: 2.7382
Epoch 8/300, Train Loss: 1.9294, Val Loss: 3.0242
Epoch 9/300, Train Loss: 2.8594, Val Loss: 10.0873
Epoch 10/300, Train Loss: 2.8568, Val Loss: 6.1491
Epoch 11/300, Train Loss: 2.4689, Val Loss: 6.0012
Epoch 12/300, Train Loss: 4.5630, Val Loss: 2.7358
Epoch 13/300, Train Loss: 3.5517, Val Loss: 3.7139
Epoch 14/300, Train Loss: 2.8601, Val Loss: 0.9097
Epoch 15/300, Train Loss: 1.9933, Val Loss: 0.1933
Epoch 16/300, Train Loss: 2.5146, Val Loss: 4.4841
Epoch 17/300, Train Loss: 1.9250, Val Loss: 2.0056
Epoch 18/300, Train Loss: 2.0134, Val Loss: 1.3290
Epoch 19/300, Train Loss: 1.5344, Val Loss: 1.1432
Epoch 20/300, Train Loss: 1.0864, Val Loss: 2.9209
Epoch 21/300, Train Loss: 1.4515, Val 

...

Epoch 2/300, Train Loss: 4.2490, Val Loss: 5.7513
Epoch 3/300, Train Loss: 4.2609, Val Loss: 5.9564
Epoch 4/300, Train Loss: 4.2609, Val Loss: 6.0219
Epoch 5/300, Train Loss: 4.4294, Val Loss: 6.4089
Epoch 6/300, Train Loss: 4.4616, Val Loss: 6.1401
Epoch 7/300, Train Loss: 4.1788, Val Loss: 5.9597
Epoch 8/300, Train Loss: 4.0798, Val Loss: 6.1943
Epoch 9/300, Train Loss: 4.2598, Val Loss: 6.0362
Epoch 10/300, Train Loss: 4.0355, Val Loss: 6.0515
Epoch 11/300, Train Loss: 4.0197, Val Loss: 5.9310
Epoch 12/300, Train Loss: 4.1505, Val Loss: 6.0700
Epoch 13/300, Train Loss: 4.1816, Val Loss: 6.0109
Epoch 14/300, Train Loss: 4.0160, Val Loss: 6.5879
Epoch 15/300, Train Loss: 4.0274, Val Loss: 6.2732
Epoch 16/300, Train Loss: 3.8952, Val Loss: 5.7852
Epoch 17/300, Train Loss: 3.9073, Val Loss: 5.9164
Epoch 18/300, Train Loss: 4.0920, Val Loss: 5.7697
Epoch 19/300, Train Loss: 3.9921, Val Loss: 5.6220
Epoch 20/300, Train Loss: 3.7903, Val Loss: 6.1689
Epoch 21/300, Train Loss: 4.0363, Val L

<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 4.5440, Val Loss: 5.7633
Epoch 3/300, Train Loss: 4.3572, Val Loss: 6.2948
Epoch 4/300, Train Loss: 4.4880, Val Loss: 6.3226
Epoch 5/300, Train Loss: 4.4715, Val Loss: 6.2879
Epoch 6/300, Train Loss: 4.3521, Val Loss: 7.3526
Epoch 7/300, Train Loss: 4.1633, Val Loss: 6.5919
Epoch 8/300, Train Loss: 4.2250, Val Loss: 6.1903
Epoch 9/300, Train Loss: 4.3669, Val Loss: 5.8742
Epoch 10/300, Train Loss: 4.1853, Val Loss: 6.4532
Epoch 11/300, Train Loss: 4.3000, Val Loss: 6.1000
Epoch 12/300, Train Loss: 4.2138, Val Loss: 6.1679
Epoch 13/300, Train Loss: 4.2129, Val Loss: 6.0453
Epoch 14/300, Train Loss: 4.3057, Val Loss: 6.0509
Epoch 15/300, Train Loss: 4.0770, Val Loss: 5.9929
Epoch 16/300, Train Loss: 4.1659, Val Loss: 5.7855
Epoch 17/300, Train Loss: 4.0697, Val Loss: 5.9836
Epoch 18/300, Train Loss: 4.0239, Val Loss: 5.9765
Epoch 19/300, Train Loss: 4.0015, Val Loss: 5.7330
Epoch 20/300, Train Loss: 4.0883, Val Loss: 5.6052
Epoch 21/300, Train Loss: 4.0682, Val L

<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 4.4527, Val Loss: 6.1904
Epoch 3/300, Train Loss: 4.2545, Val Loss: 6.0006
Epoch 4/300, Train Loss: 4.2370, Val Loss: 6.5484
Epoch 5/300, Train Loss: 4.3848, Val Loss: 6.4793
Epoch 6/300, Train Loss: 4.3808, Val Loss: 6.1314
Epoch 7/300, Train Loss: 4.3071, Val Loss: 6.0778
Epoch 8/300, Train Loss: 4.1734, Val Loss: 6.1893
Epoch 9/300, Train Loss: 4.0342, Val Loss: 6.3586
Epoch 10/300, Train Loss: 4.1278, Val Loss: 6.2359
Epoch 11/300, Train Loss: 4.1558, Val Loss: 6.1866
Epoch 12/300, Train Loss: 4.1046, Val Loss: 5.9024
Epoch 13/300, Train Loss: 3.9197, Val Loss: 6.2379
Epoch 14/300, Train Loss: 4.1858, Val Loss: 6.1297
Epoch 15/300, Train Loss: 4.1034, Val Loss: 5.9763
Epoch 16/300, Train Loss: 3.9644, Val Loss: 6.2065
Epoch 17/300, Train Loss: 3.8566, Val Loss: 5.9774
Epoch 18/300, Train Loss: 3.6948, Val Loss: 6.2081
Epoch 19/300, Train Loss: 3.7120, Val Loss: 6.1986
Epoch 20/300, Train Loss: 3.8123, Val Loss: 6.0514
Epoch 21/300, Train Loss: 3.8603, Val L

In [13]:
for id in [17,186, 110, 267]:
  train_loss_histories, val_loss_histories, train_rsme_histories, val_rsme_histories, rsme_list = run_experiment(runs=5, dataset_id=id, activation="tanh", missing_process="linear")
  print(f"Dataset {id} done")
  # print rseme_mean and std
  # convert to list
  rsme_list = [rsme.item() for rsme in rsme_list]
  print(f"Rsme mean: {np.mean(rsme_list)}")
  print(f"Rsme std: {np.std(rsme_list)}")

Run 1/5
<class 'int'>
Epoch 1/300, Train Loss: 37.3752, Val Loss: 32.9580


<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 29.8988, Val Loss: 26.0892
Epoch 3/300, Train Loss: 25.1831, Val Loss: 22.9052
Epoch 4/300, Train Loss: 22.2076, Val Loss: 20.5758
Epoch 5/300, Train Loss: 19.8936, Val Loss: 20.0776
Epoch 6/300, Train Loss: 19.6409, Val Loss: 19.6652
Epoch 7/300, Train Loss: 19.9145, Val Loss: 20.3152
Epoch 8/300, Train Loss: 19.0586, Val Loss: 18.5068
Epoch 9/300, Train Loss: 19.2869, Val Loss: 18.2925
Epoch 10/300, Train Loss: 18.3801, Val Loss: 18.3211
Epoch 11/300, Train Loss: 18.2491, Val Loss: 17.8239
Epoch 12/300, Train Loss: 17.0428, Val Loss: 17.4455
Epoch 13/300, Train Loss: 17.1641, Val Loss: 17.0807
Epoch 14/300, Train Loss: 16.9499, Val Loss: 16.7869
Epoch 15/300, Train Loss: 16.4432, Val Loss: 16.6131
Epoch 16/300, Train Loss: 16.7920, Val Loss: 17.4703
Epoch 17/300, Train Loss: 17.2729, Val Loss: 16.2517
Epoch 18/300, Train Loss: 16.3559, Val Loss: 15.5126
Epoch 19/300, Train Loss: 17.5660, Val Loss: 15.5125
Epoch 20/300, Train Loss: 15.8920, Val Loss: 15.8292
E

<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 31.6008, Val Loss: 28.0413
Epoch 3/300, Train Loss: 27.0379, Val Loss: 23.5478
Epoch 4/300, Train Loss: 22.3920, Val Loss: 21.0908
Epoch 5/300, Train Loss: 20.2942, Val Loss: 20.0878
Epoch 6/300, Train Loss: 19.5821, Val Loss: 20.3008
Epoch 7/300, Train Loss: 19.0665, Val Loss: 18.7078
Epoch 8/300, Train Loss: 18.0398, Val Loss: 19.1339
Epoch 9/300, Train Loss: 18.3244, Val Loss: 18.1319
Epoch 10/300, Train Loss: 17.2587, Val Loss: 19.9783
Epoch 11/300, Train Loss: 19.8822, Val Loss: 18.8653
Epoch 12/300, Train Loss: 18.8020, Val Loss: 17.9296
Epoch 13/300, Train Loss: 18.0497, Val Loss: 18.7364
Epoch 14/300, Train Loss: 18.5962, Val Loss: 18.1005
Epoch 15/300, Train Loss: 17.5130, Val Loss: 16.7809
Epoch 16/300, Train Loss: 17.7871, Val Loss: 16.6660
Epoch 17/300, Train Loss: 17.8078, Val Loss: 16.8105
Epoch 18/300, Train Loss: 17.4995, Val Loss: 16.2750
Epoch 19/300, Train Loss: 16.6658, Val Loss: 15.7707
Epoch 20/300, Train Loss: 16.8419, Val Loss: 17.1186
E

...

Epoch 2/300, Train Loss: 3.7344, Val Loss: 9.2001
Epoch 3/300, Train Loss: 3.3565, Val Loss: 12.1970
Epoch 4/300, Train Loss: 3.3914, Val Loss: 15.9134
Epoch 5/300, Train Loss: 3.1937, Val Loss: 27.8151
Epoch 6/300, Train Loss: 3.0169, Val Loss: 33.2949
Epoch 7/300, Train Loss: 3.0121, Val Loss: 46.5331
Epoch 8/300, Train Loss: 2.9135, Val Loss: 49.9771
Epoch 9/300, Train Loss: 2.9799, Val Loss: 52.0002
Epoch 10/300, Train Loss: 2.9751, Val Loss: 49.9680
Epoch 11/300, Train Loss: 3.1207, Val Loss: 42.0986
Epoch 12/300, Train Loss: 2.8314, Val Loss: 39.7434
Epoch 13/300, Train Loss: 2.9397, Val Loss: 33.4078
Epoch 14/300, Train Loss: 2.8330, Val Loss: 49.5989
Epoch 15/300, Train Loss: 2.8742, Val Loss: 29.1867
Epoch 16/300, Train Loss: 2.8330, Val Loss: 41.5335
Epoch 17/300, Train Loss: 2.8224, Val Loss: 46.5728
Epoch 18/300, Train Loss: 2.9463, Val Loss: 22.9209
Epoch 19/300, Train Loss: 2.7794, Val Loss: 30.6119
Epoch 20/300, Train Loss: 2.7562, Val Loss: 28.7299
Epoch 21/300, Train L

<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 4.3388, Val Loss: 4.2737
Epoch 3/300, Train Loss: 4.1817, Val Loss: 4.1112
Epoch 4/300, Train Loss: 3.9704, Val Loss: 3.9902
Epoch 5/300, Train Loss: 3.7302, Val Loss: 3.7542
Epoch 6/300, Train Loss: 3.7012, Val Loss: 3.6351
Epoch 7/300, Train Loss: 3.6062, Val Loss: 3.5102
Epoch 8/300, Train Loss: 3.5199, Val Loss: 3.2670
Epoch 9/300, Train Loss: 3.4733, Val Loss: 3.3616
Epoch 10/300, Train Loss: 3.3026, Val Loss: 3.5056
Epoch 11/300, Train Loss: 3.3448, Val Loss: 3.1312
Epoch 12/300, Train Loss: 3.2635, Val Loss: 3.5638
Epoch 13/300, Train Loss: 3.1951, Val Loss: 3.1811
Epoch 14/300, Train Loss: 3.1983, Val Loss: 3.1423
Epoch 15/300, Train Loss: 3.1722, Val Loss: 3.0428
Epoch 16/300, Train Loss: 3.1758, Val Loss: 3.1939
Epoch 17/300, Train Loss: 3.1739, Val Loss: 3.2086
Epoch 18/300, Train Loss: 3.1197, Val Loss: 3.0450
Epoch 19/300, Train Loss: 3.0934, Val Loss: 3.1946
Epoch 20/300, Train Loss: 3.0518, Val Loss: 3.2135
Epoch 21/300, Train Loss: 3.0838, Val L

<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 3.5140, Val Loss: 3.9567
Epoch 3/300, Train Loss: 3.1962, Val Loss: 4.0962
Epoch 4/300, Train Loss: 3.1117, Val Loss: 4.1879
Epoch 5/300, Train Loss: 3.0545, Val Loss: 3.8862
Epoch 6/300, Train Loss: 3.0358, Val Loss: 4.3426
Epoch 7/300, Train Loss: 2.9336, Val Loss: 4.1207
Epoch 8/300, Train Loss: 3.0032, Val Loss: 4.1318
Epoch 9/300, Train Loss: 3.0304, Val Loss: 4.5873
Epoch 10/300, Train Loss: 2.9005, Val Loss: 4.9281
Epoch 11/300, Train Loss: 2.9445, Val Loss: 4.2405
Epoch 12/300, Train Loss: 3.0133, Val Loss: 4.3339
Epoch 13/300, Train Loss: 3.1510, Val Loss: 4.2074
Epoch 14/300, Train Loss: 2.9389, Val Loss: 4.0667
Epoch 15/300, Train Loss: 2.8716, Val Loss: 3.9084
Epoch 16/300, Train Loss: 2.8284, Val Loss: 4.4680
Epoch 17/300, Train Loss: 3.0062, Val Loss: 4.0043
Epoch 18/300, Train Loss: 2.9487, Val Loss: 5.2779
Epoch 19/300, Train Loss: 2.9403, Val Loss: 3.7710
Epoch 20/300, Train Loss: 2.7667, Val Loss: 4.9189
Epoch 21/300, Train Loss: 2.8304, Val L

In [10]:
for id in [17,186, 110, 267]:
  train_loss_histories, val_loss_histories, train_rsme_histories, val_rsme_histories, rsme_list = run_experiment(runs=5, dataset_id=id, activation="tanh", missing_process="non_linear")
  print(f"Dataset {id} done")
  # print rseme_mean and std
  # convert to list
  rsme_list = [rsme.item() for rsme in rsme_list]
  print(f"Rsme mean: {np.mean(rsme_list)}")
  print(f"Rsme std: {np.std(rsme_list)}")

Run 1/5
<class 'int'>
Epoch 1/300, Train Loss: 37.1496, Val Loss: 29.4167


<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 24.5084, Val Loss: 21.3628
Epoch 3/300, Train Loss: 21.8706, Val Loss: 19.6173
Epoch 4/300, Train Loss: 19.4810, Val Loss: 18.9397
Epoch 5/300, Train Loss: 19.8092, Val Loss: 18.0730
Epoch 6/300, Train Loss: 18.7557, Val Loss: 18.2272
Epoch 7/300, Train Loss: 18.4548, Val Loss: 17.0694
Epoch 8/300, Train Loss: 17.5787, Val Loss: 17.7592
Epoch 9/300, Train Loss: 16.6610, Val Loss: 14.1684
Epoch 10/300, Train Loss: 15.0172, Val Loss: 12.0725
Epoch 11/300, Train Loss: 12.0308, Val Loss: 11.6272
Epoch 12/300, Train Loss: 9.6923, Val Loss: 9.8241
Epoch 13/300, Train Loss: 8.4043, Val Loss: 7.6682
Epoch 14/300, Train Loss: 7.1236, Val Loss: 5.5368
Epoch 15/300, Train Loss: 6.0184, Val Loss: 5.4297
Epoch 16/300, Train Loss: 6.0433, Val Loss: 6.3334
Epoch 17/300, Train Loss: 4.2253, Val Loss: 4.2172
Epoch 18/300, Train Loss: 2.6101, Val Loss: 3.6082
Epoch 19/300, Train Loss: 1.9536, Val Loss: 4.1883
Epoch 20/300, Train Loss: 2.4720, Val Loss: 1.7387
Epoch 21/300, Train

<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 24.9019, Val Loss: 21.8570
Epoch 3/300, Train Loss: 21.1378, Val Loss: 20.7499
Epoch 4/300, Train Loss: 20.7294, Val Loss: 20.1708
Epoch 5/300, Train Loss: 19.2281, Val Loss: 18.9828
Epoch 6/300, Train Loss: 18.4653, Val Loss: 19.0967
Epoch 7/300, Train Loss: 18.8691, Val Loss: 18.7872
Epoch 8/300, Train Loss: 18.1353, Val Loss: 17.0981
Epoch 9/300, Train Loss: 18.2821, Val Loss: 17.6550
Epoch 10/300, Train Loss: 17.2901, Val Loss: 17.0071
Epoch 11/300, Train Loss: 17.7896, Val Loss: 16.7040
Epoch 12/300, Train Loss: 16.1080, Val Loss: 15.8697
Epoch 13/300, Train Loss: 16.4834, Val Loss: 14.5155
Epoch 14/300, Train Loss: 14.0521, Val Loss: 11.4497
Epoch 15/300, Train Loss: 10.8059, Val Loss: 12.2138
Epoch 16/300, Train Loss: 10.7101, Val Loss: 8.1204
Epoch 17/300, Train Loss: 8.9959, Val Loss: 8.0689
Epoch 18/300, Train Loss: 7.0189, Val Loss: 5.6543
Epoch 19/300, Train Loss: 6.4333, Val Loss: 7.2783
Epoch 20/300, Train Loss: 6.2669, Val Loss: 4.4623
Epoch 21/3

...

Epoch 2/300, Train Loss: 3.4337, Val Loss: 4.5717
Epoch 3/300, Train Loss: 3.0256, Val Loss: 5.6891
Epoch 4/300, Train Loss: 3.1079, Val Loss: 5.7727
Epoch 5/300, Train Loss: 3.1587, Val Loss: 5.1564
Epoch 6/300, Train Loss: 3.0017, Val Loss: 5.4430
Epoch 7/300, Train Loss: 2.9231, Val Loss: 5.6925
Epoch 8/300, Train Loss: 3.0926, Val Loss: 4.4912
Epoch 9/300, Train Loss: 2.7773, Val Loss: 5.5109
Epoch 10/300, Train Loss: 2.8305, Val Loss: 4.6780
Epoch 11/300, Train Loss: 2.8707, Val Loss: 5.0271
Epoch 12/300, Train Loss: 2.8740, Val Loss: 4.6937
Epoch 13/300, Train Loss: 2.8023, Val Loss: 5.4428
Epoch 14/300, Train Loss: 2.8152, Val Loss: 5.2164
Epoch 15/300, Train Loss: 2.8608, Val Loss: 4.5130
Epoch 16/300, Train Loss: 2.7443, Val Loss: 4.7548
Epoch 17/300, Train Loss: 2.7957, Val Loss: 4.8417
Epoch 18/300, Train Loss: 2.8217, Val Loss: 5.0827
Epoch 19/300, Train Loss: 2.7278, Val Loss: 5.3780
Epoch 20/300, Train Loss: 2.8977, Val Loss: 4.6384
Epoch 21/300, Train Loss: 2.7034, Val L

<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 3.1660, Val Loss: 3.8218
Epoch 3/300, Train Loss: 3.0361, Val Loss: 4.4274
Epoch 4/300, Train Loss: 3.0261, Val Loss: 4.1022
Epoch 5/300, Train Loss: 2.9159, Val Loss: 3.9717
Epoch 6/300, Train Loss: 2.9977, Val Loss: 4.0993
Epoch 7/300, Train Loss: 3.0176, Val Loss: 3.8857
Epoch 8/300, Train Loss: 2.9252, Val Loss: 4.1150
Epoch 9/300, Train Loss: 2.9982, Val Loss: 3.8005
Epoch 10/300, Train Loss: 2.9029, Val Loss: 3.9617
Epoch 11/300, Train Loss: 2.8930, Val Loss: 4.0900
Epoch 12/300, Train Loss: 2.9948, Val Loss: 3.9277
Epoch 13/300, Train Loss: 2.8188, Val Loss: 4.6471
Epoch 14/300, Train Loss: 3.0128, Val Loss: 4.0358
Epoch 15/300, Train Loss: 2.7423, Val Loss: 4.0144
Epoch 16/300, Train Loss: 2.8245, Val Loss: 4.1152
Epoch 17/300, Train Loss: 2.8382, Val Loss: 5.4214
Epoch 18/300, Train Loss: 2.7773, Val Loss: 4.2304
Epoch 19/300, Train Loss: 2.8040, Val Loss: 4.0850
Epoch 20/300, Train Loss: 2.7002, Val Loss: 4.7546
Epoch 21/300, Train Loss: 2.8087, Val L

<ipython-input-5-1ea74a910f22>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=device)
<ipython-input-5-1ea74a910f22>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  s = torch.tensor(s, dtype=torch.float32, device=device)


Epoch 2/300, Train Loss: 3.1357, Val Loss: 6.7588
Epoch 3/300, Train Loss: 3.4213, Val Loss: 5.0003
Epoch 4/300, Train Loss: 3.1017, Val Loss: 4.8252
Epoch 5/300, Train Loss: 3.0296, Val Loss: 4.6828
Epoch 6/300, Train Loss: 2.8739, Val Loss: 5.0873
Epoch 7/300, Train Loss: 2.9703, Val Loss: 4.6345
Epoch 8/300, Train Loss: 2.8503, Val Loss: 5.9661
Epoch 9/300, Train Loss: 2.9068, Val Loss: 6.1431
Epoch 10/300, Train Loss: 2.9758, Val Loss: 4.4064
Epoch 11/300, Train Loss: 2.7623, Val Loss: 4.8063
Epoch 12/300, Train Loss: 2.8320, Val Loss: 6.2660
Epoch 13/300, Train Loss: 2.9022, Val Loss: 5.7334
Epoch 14/300, Train Loss: 2.7654, Val Loss: 5.5102
Epoch 15/300, Train Loss: 2.6552, Val Loss: 7.8718
Epoch 16/300, Train Loss: 2.7576, Val Loss: 8.0751
Epoch 17/300, Train Loss: 2.7807, Val Loss: 6.6901
Epoch 18/300, Train Loss: 2.6969, Val Loss: 5.4345
Epoch 19/300, Train Loss: 2.8663, Val Loss: 4.9550
Epoch 20/300, Train Loss: 2.8028, Val Loss: 5.2815
Epoch 21/300, Train Loss: 2.6826, Val L